In [1]:
# pip install librosa

In [2]:
# pip install soundfile

In [3]:
# pip install accelerate -U

In [4]:
# pip install wandb

In [5]:
from datasets import load_dataset, Audio

In [6]:
data = load_dataset("train/train/audio", name="en-US", split="train")

FileNotFoundError: Couldn't find a dataset script at c:\Users\User\Documents\Studia\dlm-2\train\train\audio\audio.py or any data file in the same directory.

In [ ]:
data = data.train_test_split(test_size=0.2)

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['audio', 'label'],
        num_rows: 51781
    })
    test: Dataset({
        features: ['audio', 'label'],
        num_rows: 12946
    })
})

In [ ]:
data["train"][0]

{'audio': {'path': 'c:\\Users\\User\\Documents\\Studia\\dlm-2\\train\\train\\audio\\on\\55d3725a_nohash_0.wav',
  'array': array([-0.00115967, -0.00137329, -0.00152588, ...,  0.00119019,
          0.00125122,  0.0015564 ]),
  'sampling_rate': 16000},
 'label': 17}

In [ ]:
labels = data["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [ ]:
id2label[str(2)]

'bird'

In [ ]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

e:\conda\Lib\site-packages\transformers\configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [ ]:
data = data.cast_column("audio", Audio(sampling_rate=16_000))
data["train"][0]

{'audio': {'path': 'c:\\Users\\User\\Documents\\Studia\\dlm-2\\train\\train\\audio\\on\\55d3725a_nohash_0.wav',
  'array': array([-0.00115967, -0.00137329, -0.00152588, ...,  0.00119019,
          0.00125122,  0.0015564 ]),
  'sampling_rate': 16000},
 'label': 17}

In [ ]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000, truncation=True
    )
    return inputs

In [ ]:
encoded_data = data.map(preprocess_function, remove_columns="audio", batched=True)

Map:   0%|          | 0/51781 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base", num_labels=num_labels, label2id=label2id, id2label=id2label
)

e:\conda\Lib\site-packages\transformers\configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'classifier.weight', 'classifier.bias', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# wandab login (w cmd)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
training_args = TrainingArguments(
    output_dir="audio_model_v2",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_data["train"].with_format("torch"),
    eval_dataset=encoded_data["test"].with_format("torch"),
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

e:\conda\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(


  0%|          | 0/4040 [00:00<?, ?it/s]

e:\conda\Lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 3.4252, 'learning_rate': 7.425742574257426e-07, 'epoch': 0.02}
{'loss': 3.4229, 'learning_rate': 1.4851485148514852e-06, 'epoch': 0.05}
{'loss': 3.423, 'learning_rate': 2.227722772277228e-06, 'epoch': 0.07}
{'loss': 3.4205, 'learning_rate': 2.9702970297029703e-06, 'epoch': 0.1}
{'loss': 3.4179, 'learning_rate': 3.7128712871287128e-06, 'epoch': 0.12}
{'loss': 3.412, 'learning_rate': 4.455445544554456e-06, 'epoch': 0.15}
{'loss': 3.4074, 'learning_rate': 5.198019801980198e-06, 'epoch': 0.17}
{'loss': 3.3967, 'learning_rate': 5.940594059405941e-06, 'epoch': 0.2}
{'loss': 3.3887, 'learning_rate': 6.6831683168316835e-06, 'epoch': 0.22}
{'loss': 3.374, 'learning_rate': 7.4257425742574256e-06, 'epoch': 0.25}
{'loss': 3.3522, 'learning_rate': 8.168316831683168e-06, 'epoch': 0.27}
{'loss': 3.3166, 'learning_rate': 8.910891089108911e-06, 'epoch': 0.3}
{'loss': 3.2583, 'learning_rate': 9.653465346534654e-06, 'epoch': 0.32}
{'loss': 3.1803, 'learning_rate': 1.0396039603960395e-05, 'epoch'

  0%|          | 0/405 [00:00<?, ?it/s]

...

{'loss': 0.6555, 'learning_rate': 2.665016501650165e-05, 'epoch': 2.0}
{'loss': 0.5681, 'learning_rate': 2.656765676567657e-05, 'epoch': 2.03}
{'loss': 0.5954, 'learning_rate': 2.6485148514851484e-05, 'epoch': 2.05}
{'loss': 0.5778, 'learning_rate': 2.6402640264026404e-05, 'epoch': 2.08}
{'loss': 0.6063, 'learning_rate': 2.632013201320132e-05, 'epoch': 2.1}
{'loss': 0.4851, 'learning_rate': 2.623762376237624e-05, 'epoch': 2.12}
{'loss': 0.5138, 'learning_rate': 2.6155115511551156e-05, 'epoch': 2.15}
{'loss': 0.5294, 'learning_rate': 2.6072607260726073e-05, 'epoch': 2.17}
{'loss': 0.5302, 'learning_rate': 2.599009900990099e-05, 'epoch': 2.2}
{'loss': 0.518, 'learning_rate': 2.590759075907591e-05, 'epoch': 2.22}
{'loss': 0.4855, 'learning_rate': 2.5825082508250826e-05, 'epoch': 2.25}
{'loss': 0.4805, 'learning_rate': 2.5742574257425742e-05, 'epoch': 2.27}
{'loss': 0.4888, 'learning_rate': 2.5660066006600662e-05, 'epoch': 2.3}
{'loss': 0.523, 'learning_rate': 2.5577557755775575e-05, 'epoc

  0%|          | 0/405 [00:00<?, ?it/s]

{'eval_loss': 0.20769630372524261, 'eval_accuracy': 0.9744322570678202, 'eval_runtime': 73.104, 'eval_samples_per_second': 177.09, 'eval_steps_per_second': 5.54, 'epoch': 3.0}


e:\conda\Lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.3623, 'learning_rate': 2.3267326732673265e-05, 'epoch': 3.01}
{'loss': 0.4025, 'learning_rate': 2.3184818481848185e-05, 'epoch': 3.04}
{'loss': 0.3332, 'learning_rate': 2.31023102310231e-05, 'epoch': 3.06}
{'loss': 0.3648, 'learning_rate': 2.301980198019802e-05, 'epoch': 3.09}
{'loss': 0.3153, 'learning_rate': 2.2937293729372938e-05, 'epoch': 3.11}
{'loss': 0.3694, 'learning_rate': 2.2854785478547858e-05, 'epoch': 3.14}
{'loss': 0.372, 'learning_rate': 2.277227722772277e-05, 'epoch': 3.16}
{'loss': 0.3582, 'learning_rate': 2.268976897689769e-05, 'epoch': 3.19}
{'loss': 0.323, 'learning_rate': 2.2607260726072607e-05, 'epoch': 3.21}
{'loss': 0.3633, 'learning_rate': 2.2524752475247527e-05, 'epoch': 3.24}
{'loss': 0.3318, 'learning_rate': 2.2442244224422444e-05, 'epoch': 3.26}
{'loss': 0.3298, 'learning_rate': 2.235973597359736e-05, 'epoch': 3.29}
{'loss': 0.3636, 'learning_rate': 2.2277227722772277e-05, 'epoch': 3.31}
{'loss': 0.3533, 'learning_rate': 2.2194719471947197e-05, '